# **Construcción de un dashboard con Pandas y Panel**

En este ejercicio vamos a construir un panel interactivo en Python con Panel.

!['CO2 Dashboard'](images/co2-dashboard.png)

## **Paso 0. Importar los datos del CSV**


In [ ]:
# Importar dependencias
# Importar pandas 
# Importar numpy
import pandas as pd
import numpy as np
import panel as pn
import panel.widgets as pnw
pn.extension('tabulator')
import hvplot.pandas

# Leer los datos del CSV

In [14]:
# MEJORA - Meter datos en caché para mejorar el rendimiento del dashboard
if 'data' not in pn.state.cache.keys():
    # Leer datos del CSV
    df = pd.read_csv("owid-co2-data.csv")
    pn.state.cache["data"] = df.copy()
else:
    df = pn.state.cache["data"]
    

## **Paso 1. Revisar los datos y hacer limpieza**

In [ ]:
# Inspeccionar datos importados
df.info()

In [15]:
# Ejecutar tareas de limpieza

# Rellenar celdas vacías con 0s y crear la columna GDP per cápita

df.fillna (0, inplace = True)

df["GDP"] = df["gdp"] / df["population"]

df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share,GDP
0,Afghanistan,1949,AFG,7624058.0,0.000000e+00,0.000,0.000,0.015,0.000,0.00,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000,0.00,0.000000
1,Afghanistan,1950,AFG,7752117.0,9.421400e+09,0.000,0.000,0.084,0.070,475.00,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000,0.00,1215.332543
2,Afghanistan,1951,AFG,7840151.0,9.692280e+09,0.000,0.000,0.092,0.007,8.70,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000,0.00,1236.236369
3,Afghanistan,1952,AFG,7935996.0,1.001733e+10,0.000,0.000,0.092,0.000,0.00,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000,0.00,1262.264378
4,Afghanistan,1953,AFG,8039684.0,1.063052e+10,0.000,0.000,0.106,0.015,16.00,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000,0.00,1322.255925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26003,Zimbabwe,2016,ZWE,14030338.0,2.096179e+10,0.639,0.046,10.738,-1.488,-12.17,...,0.0,0.0,0.0,0.03,0.0,115.92,28.53,1.415,13.18,1494.033433
26004,Zimbabwe,2017,ZWE,14236599.0,2.194784e+10,0.678,0.048,9.582,-1.156,-10.77,...,0.0,0.0,0.0,0.03,0.0,115.59,28.30,1.666,17.39,1541.648846
26005,Zimbabwe,2018,ZWE,14438812.0,2.271535e+10,0.697,0.048,11.854,2.273,23.72,...,0.0,0.0,0.0,0.03,0.0,118.22,30.83,1.308,11.04,1573.214825
26006,Zimbabwe,2019,ZWE,14645473.0,0.000000e+00,0.697,0.048,10.949,-0.905,-7.64,...,0.0,0.0,0.0,0.03,0.0,117.96,30.53,1.473,13.45,0.000000


In [16]:
# Hacer el pipeline del DataFrame interactivo
idf = df.interactive()
idf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26008 entries, 0 to 26007
Data columns (total 61 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              26008 non-null  object 
 1   year                                 26008 non-null  int64  
 2   iso_code                             26008 non-null  object 
 3   population                           26008 non-null  float64
 4   gdp                                  26008 non-null  float64
 5   cement_co2                           26008 non-null  float64
 6   cement_co2_per_capita                26008 non-null  float64
 7   co2                                  26008 non-null  float64
 8   co2_growth_abs                       26008 non-null  float64
 9   co2_growth_prct                      26008 non-null  float64
 10  co2_per_capita                       26008 non-null  float64
 11  co2_per_gdp                 

## **Paso 2. Crear gráfica de "Emisiones de CO2 por continente"**

In [17]:
# Crear el panel de controles
year_slider = pnw.IntSlider (name = "Year", start = df["year"].min(), end = df["year"].max())
year_slider

IntSlider(end=2020, name='Year', start=1750, value=1750)

In [18]:
# Crear los botones para las mediciones de CO2
yaxis_co2 = pnw.RadioButtonGroup(name = "botones CO2", options = ["co2", "co2_per_capita"], button_type="success")
yaxis_co2


RadioButtonGroup(button_type='success', name='botones CO2', options=['co2', 'co2_per_capita'], value='co2')

In [19]:
# Crear pipeline
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline =        ( idf[(idf.year <= year_slider) & (idf.country.isin(continents))]
    .groupby(["country", "year"]) [yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = "year")
    .reset_index (drop = True)
    )
co2_pipeline

In [20]:
# Crear gráfica
co2_plot = co2_pipeline.hvplot (x= "year", by = "country", y = yaxis_co2, line_width=5)

co2_plot

## **Paso 3. Crear tabla "Emisiones de CO2 por continente"**

In [ ]:
co2_table = 

## **Paso 4. Crear gráfico de dispersión "CO2 vs. GDP"**

In [ ]:
co2_vs_gdp_scatterplot = ...

## **Paso 5. Crear gráfico de barras con "Fuentes de CO2 por continente"**

In [ ]:
yaxis_co2_source = ...

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = ...

co2_source_bar_plot = ...

## **Paso final. Montar el dashboard**

In [ ]:
# Diseñar distribución con Template
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             pn.pane.PNG('images/climate_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

